In [1]:
import os

import numpy as np
import pandas as pd

from npfd.models.HTK.htktools import clean_dir
from npfd.data.size_distribution import decimalDOY2datetime as d2d

import matplotlib.pyplot as plt

EVENT_CLASSIFICATION_NUK_PATH =  '../data/raw/dmps/event_classification.nuk'
EVENT_CLASSIFICATION_OUT_PATH =  '../data/raw/dmps/event_classification.csv'

El archivo .nuk fue generado con el codigo de Matlab.

Las etiquetas pueden ser:

 Bandera | Descripción
 --------|------------
    0    | No hay evento de FNP o no hay dato
    1    | Hay un evento cuantificable
    2    | ?
    3    | ?


In [2]:
dmps_labels = pd.read_csv(EVENT_CLASSIFICATION_NUK_PATH, delim_whitespace=True,
                          names=['doy', 'event_type_1', 'start_event_1', 'end_event_1', 'event_type_2',
                                 'start_event_2', 'end_event_2'], na_values='NaN')

dmps_labels.index = d2d(dmps_labels['doy'])
dmps_labels = dmps_labels.reindex(pd.date_range('1/1/2017', '1/1/2018'))

labels = pd.DataFrame(index=pd.date_range(start='1/1/2017', end='1/1/2018', freq='10T'), columns=['label'])

In [3]:
dmps_labels['start_event_1'][~dmps_labels['start_event_1'].isna()] = dmps_labels['start_event_1'][~dmps_labels['start_event_1'].isna()].apply(d2d)
dmps_labels['end_event_1'][~dmps_labels['end_event_1'].isna()] = dmps_labels['end_event_1'][~dmps_labels['end_event_1'].isna()].apply(d2d)

dmps_labels['start_event_2'][~dmps_labels['start_event_2'].isna()] = dmps_labels['start_event_2'][~dmps_labels['start_event_2'].isna()].apply(d2d)
dmps_labels['end_event_2'][~dmps_labels['end_event_2'].isna()] = dmps_labels['end_event_2'][~dmps_labels['end_event_2'].isna()].apply(d2d)

/home/gfogwil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gfogwil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/gfogwil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
dmps_labels.head()

,doy,event_type_1,start_event_1,end_event_1,event_type_2,start_event_2,end_event_2
2017-01-01,1.0,3.0,2017-01-01 22:07:15.484800,2017-01-02 00:13:03.872640,NaN,NaT,NaT
2017-01-02,2.0,3.0,2017-01-01 23:42:02.073600,2017-01-02 03:25:54.698880,3.0,2017-01-02 15:19:00.117120,2017-01-02 20:31:19.464960
2017-01-03,3.0,0.0,NaT,NaT,NaN,NaT,NaT
2017-01-04,4.0,3.0,2017-01-04 10:59:11.630400,2017-01-04 11:51:42.491520,3.0,2017-01-04 19:33:16.931520,2017-01-04 20:25:47.792640
2017-01-05,5.0,2.0,2017-01-05 00:23:42.584640,2017-01-05 01:34:21.290880,3.0,2017-01-05 05:04:21.288000,2017-01-05 05:44:01.936320


In [5]:
for i, day in dmps_labels.iterrows():
    if pd.isnull(day['start_event_1']) or day['event_type_1'] ==4:
        continue
        
    else:
        labels[day['start_event_1']:day['end_event_1']] = day['event_type_1']

    if pd.isnull(day['start_event_2']):
        continue
        
    else:
        labels[day['start_event_2']:day['end_event_2']] = day['event_type_2']

labels.head()

,label
2017-01-01 00:00:00,NaN
2017-01-01 00:10:00,NaN
2017-01-01 00:20:00,NaN
2017-01-01 00:30:00,NaN
2017-01-01 00:40:00,NaN


In [6]:
labels['label'].isna().count()

52561

In [7]:
print(labels[labels['label'] == 0].count())
print(labels['label'].isna().count())
print(labels[labels['label'] == 1].count())
print(labels[labels['label'] == 2].count())
print(labels[labels['label'] == 3].count())

label    0
dtype: int64
52561
label    388
dtype: int64
label    1648
dtype: int64
label    2827
dtype: int64


In [8]:
labels = labels.replace(0, 'ne')
labels = labels.replace(np.nan, 'ne')
labels = labels.replace(1, 'e')
labels = labels.replace(2, 'ne')
labels = labels.replace(3, 'ne')

In [9]:
labels.to_csv(EVENT_CLASSIFICATION_OUT_PATH)